<a href="https://colab.research.google.com/github/koglak/SWE573/blob/main/SWE599_Flight_Price_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import selenium.common.exceptions as selexcept
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import logging

import pandas as pd
from datetime import datetime
import os.path
import re
import sys
import glob
import time


ModuleNotFoundError: ignored

In [ ]:
date_list = pd.date_range(start='14.11.2022',end='14.12.2022').strftime('%d.%m.%Y').tolist()

airports=[["izmir", "adnan-menderes-havalimani","adb"],
          ["ankara", "esenboga-havalimani","esb"],
          ["antalya","havalimani","ayt"],
          ["adana","sakir-pasa-havalimani","ada"],
          ["bodrum","milas-havalimani","bjv"],
          ["dalaman","havalimani","dlm"],
          ["trabzon","havalimani","tzx"],
          ["gaziantep","oguzeli-havalimani","gzt"],
          ["diyarbakir","havalimani","diy"],
          ["hatay","havalimani","hty"],
          ["kars","havalimani","ksy"],
          ["erzurum","havalimani","erz"],
          ["mardin","havalimani","mqm"],
]

In [ ]:
URLs=[]
file_name=[]
for date_len in range(0,len(date_list)):
    for dep in range(0,len(airports)):
        url= "https://www.enuygun.com/ucak-bileti/istanbul-"+ airports[dep][0]+"-"+airports[dep][1]+"-ista-"+airports[dep][2]+"/?gidis="+date_list[date_len]+"&yetiskin=1&sinif=ekonomi&aktarmasiz=1&save=1&geotrip=domestic&trip=domestic"
        URLs.append(url)
        file_name.append(airports[dep][0]+"-"+date_list[date_len].replace(".","-"))

In [ ]:
count=0
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    for num in range(0,len(URLs)):
        #run driver on Chrome
        with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
            driver.get(URLs[num])
            time.sleep(2)
        
            #get flights
            flights= driver.find_elements(By.CLASS_NAME, "flight-summary-infos")
            flights_list=[]

            #push each flight info to flights_list
            for flight in flights: 
                checked_date=datetime.now().strftime("%d/%m/%Y %H:%M")
                company= flight.find_elements(By.CLASS_NAME, "summary-marketing-airlines")[0].text
                departure_airport= flight.find_elements(By.CLASS_NAME, "itemAirport")[0].text
                arrival_airport= flight.find_elements(By.CLASS_NAME, "itemAirport")[2].text
                departure_time= flight.find_elements(By.CLASS_NAME, "flight-departure-time")[0].text
                arrival_time= flight.find_elements(By.CLASS_NAME, "flight-arrival-time")[0].text
                departure_date= flight.find_elements(By.XPATH, "//*[@id='SearchRoot']/div/div[1]/div/div[1]/div")[0].text.split('|')[1]
                duration= flight.find_elements(By.XPATH, "//*[@id='SearchRoot']/div/div[2]/div[2]/div/div[2]/div[1]/div/div[2]/div[1]/div/div[1]/div[2]/div/div[3]/div[2]/span")[0].text
                price= flight.find_elements(By.CLASS_NAME, "money-int")[0].text + flight.find_elements(By.CLASS_NAME, "money-decimal")[0].text

                flight_item={
                    'checked_date':checked_date,
                    'company':company,
                    'departure_airport':departure_airport,
                    'arrival_airport': arrival_airport,
                    'departure_time': departure_time,
                    'arrival_time': arrival_time,
                    'departure_date':departure_date,
                    'duration': duration,
                    'price':price,
                }

                flights_list.append(flight_item)

                # close browser

                #convert dataframe
                df=pd.DataFrame(flights_list)
                #save to local file
                df.to_csv(r'C:\Users\kogla\OneDrive\Masaüstü\SWE599\dataset\{}.csv'.format(file_name[num]), index = True, header=True)

                count=+1
            
except Exception as e:
    print(count)
    logging.error(e, exc_info=True)
    pass
    